In [ ]:
# pip install webdriver_manager

In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

In [2]:
# Set up Splinter: set up the executable path, then set up the URL
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/soha/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


### The Article

In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)
## In the line above, we are:
## Searching for elt.s with a sp. combination of tag (div) and attribute (list_text);
## (Eg: 'div.list_text' exists as <div class="list_text"> in HTML)
## and, we are telling the browser to wait 1 second before searching for components -
## this gives dynmaic pages a little while to load, esp if they are image-heavy.

True

In [4]:
# Set up the HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

## NOTE: Notice how we've assigned slide_elem as the variable to look for the <div /> tag and its descendent 
## (the other tags within the <div /> element)? This is our parent element. This means that this element holds 
## all of the other elements within it, and we'll reference it when we want to filter search results even further. 
## The . is used for selecting classes, such as list_text, so the code 'div.list_text' pinpoints the <div /> tag 
## with the class of list_text. CSS works from right to left, such as returning the last item on the list instead 
## of the first. Because of this, when using select_one, the first matching element returned will be a <li /> 
## element with a class of slide and all nested elements within it.

In [5]:
# Scrape the title of the most recent article published
slide_elem.find('div', class_='content_title')
## This, however, prints the title Along with the extra HTML stuff.
## We only require the text.

<div class="content_title">A New Video Captures the Science of NASA's Perseverance Mars Rover</div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title
## This returns only the text of the elt.,
## which in our case is the title of the news article. 

"A New Video Captures the Science of NASA's Perseverance Mars Rover"

In [7]:
# NOTE: 
## .find() is used when we want only the first class and attribute we've specified.
## .find_all() is used when we want to retrieve all of the tags and attributes.

In [8]:
# Scrape the summary text for the most recent article published
## Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'With a targeted launch date of July 30, the next robotic scientist NASA is sending to the to the Red Planet has big ambitions.'

### Featured Images

In [9]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

## The index [1] stipulates that the second button be clicked (which is our required button).

In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

## An img tag is nested within this HTML, so we've included it.
## .get('src') pulls the link to the image.

'image/featured/mars3.jpg'

In [13]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

## A NOTE on f-strings: 
## We're using an f-string for this print statement because it's a cleaner way to create print statements; 
## they're also evaluated at run-time. This means that it, and the variable it holds, doesn't exist until the 
## code is executed and the values are not constant. This works well for our scraping app because the data 
## we're scraping is live and will be updated frequently.

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Table of Facts

In [14]:
# Converting HTML-coded table into a DataFrame
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

# The above mentioned steps do the following:
## 1st line: The Pandas function read_html() specifically searches for and returns a list of tables found in the 
## HTML. By specifying an index of 0, we're telling Pandas to pull only the first table it encounters, or the first
## item in the list. Then, it turns the table into a DataFrame.
## 2nd line: Here, we assign columns to the new DataFrame for additional clarity.
## 3rd line: By using the .set_index() function, we're turning the Description column into the DataFrame's index. 
## inplace=True means that the updated index will remain in place, without having to reassign the DataFrame to a 
## new variable.

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
# Converting the DataFrame we created into HTML-ready code
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [16]:
# Ending the automated browsing session
browser.quit()